In [1]:
import numpy as np
from pcm_construction.bbcodes import bbpcm
from src.noise.data_qubit import depolarizing_round, destructive_error
from src.closed_branch_decoder import CB_decoder
from scipy.linalg import null_space
import galois    

In [20]:
# We will first create the parity check matrix. For this example, it will be l = 6, m = 6 A_poly = 
l = 9
m = 6 
# A polynomial = x^3+y+y^2
A_poly = [[0,3], [1,1], [1,2]]
# B polynomial = y^3 + x + x^2
B_poly = [[1,3], [0,1], [0,2]]

H, Hx, Hz = bbpcm(l = l, m = m, A_poly= A_poly, B_poly = B_poly)

GF = galois.GF(2)

gf_Hx = galois.GF2(Hx.astype(int))
gf_Hz = galois.GF2(Hz.astype(int))

Hx_nullspace = gf_Hx.null_space().view(np.ndarray)
Hz_nullspace = gf_Hz.null_space().view(np.ndarray)



# TODO los errores x no deberían formar parte del nullspace the Hz ni los z del nullspace de Hx

# ps = np.linspace(0.005, 0.04, num = 8)
ps = [ 0.002]
NMC = [10**3]
n_branches = 10
n_growths = 6
name_file = f'data\data_l{l}_m{m}_ng{n_growths}_nb{n_branches}2.txt'

for index, p in enumerate(ps):
    print(f'Prob: {p}')
    model = (H, p)    
    myDecoder = CB_decoder(model, max_branches = n_branches, max_growths = n_growths, bp_reweighting= True)
    Pl = 0        
    for i in range(NMC[index]):
        print(f'Numero {i}')
        error = destructive_error(H)
        syndrome = (np.dot(H, error) % 2).astype(int)

        error_recovered = myDecoder.decode(syndrome, comments = False)

        tot_err = error_recovered ^ error
        
        if np.all(tot_err == 0):
            continue
        
        elif not np.all((np.dot(H, tot_err) % 2).astype(int) == 0):
            Pl += 1/NMC[index]
            break
        tot_errorx = tot_err[:H.shape[1]//2]
        if not np.all((np.dot(Hz_nullspace, tot_errorx) % 2).astype(int) == 0):
            Pl += 1/NMC[index]
            break
        tot_errorz = tot_err[H.shape[1]//2:]
        if not np.all((np.dot(Hx_nullspace, tot_errorz) % 2).astype(int) == 0):
            Pl += 1/NMC[index]
            break
        print('Error degenerado!')
        
    with open(name_file, 'a') as file:
        # file.write(f'{p}\t\t {Pl}\n')
        print(f'{p}\t\t {Pl}\n')

Prob: 0.002
Numero 0
Numero 1
Numero 2
Numero 3
Numero 4
Numero 5
Numero 6
Numero 7
Error degenerado!
Numero 8
Error degenerado!
Numero 9
Numero 10
Numero 11
Numero 12
Numero 13
Numero 14
Numero 15
Numero 16
Numero 17
Numero 18
Numero 19
Numero 20
Numero 21
Numero 22
Numero 23
Numero 24
Numero 25
Numero 26
Numero 27
Numero 28
Numero 29
Numero 30
Numero 31
Numero 32
Numero 33
Numero 34
Numero 35
Numero 36
Numero 37
Numero 38
Numero 39
Numero 40
Numero 41
Numero 42
Numero 43
Numero 44
Error degenerado!
Numero 45
Numero 46
Numero 47
Numero 48
Error degenerado!
Numero 49
Numero 50
Numero 51
Error degenerado!
Numero 52
Error degenerado!
Numero 53
Numero 54
Numero 55
Numero 56
Numero 57
Numero 58
Numero 59
Numero 60
Numero 61
Numero 62
Numero 63
Numero 64
Numero 65
Numero 66
Numero 67
Numero 68
Numero 69
Error degenerado!
Numero 70
Numero 71
Error degenerado!
Numero 72
Numero 73
Numero 74
Numero 75
Error degenerado!
Numero 76
Numero 77
Error degenerado!
Numero 78
Numero 79
Numero 80
Error de

In [19]:
print(np.where(error_recovered == 1)[0])
print(np.where(error == 1)[0])
print(np.where(tot_err == 1)[0])
print(np.where(syndrome == 1)[0])
print(np.where((np.dot(Hz_nullspace, tot_errorx) % 2).astype(int) == 1)[0])
error_recovered = myDecoder.decode(syndrome, comments = True)

[ 0  1  2  3  4  5 18 20 22]
[ 2  3 22]
[ 0  1  4  5 18 20]
[0 2 4]
[ 0  1  4  5 18 20]


In [16]:
myDecoder._bp.log_prob_ratios

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])